# Q1
Run process-run.py with the following flags: `-l 5:100,5:100`.
- What should the CPU utilization be (e.g., the percent of time the CPU is in use?)? 
- Why do you know this? 
- Use the -c and -p flags to see if you were right.

In [5]:
%run process-run.py -l 5:100,5:100

Produce a trace of what would happen when you run these processes:
Process 0
  cpu
  cpu
  cpu
  cpu
  cpu

Process 1
  cpu
  cpu
  cpu
  cpu
  cpu

Important behaviors:
  System will switch when the current process is FINISHED or ISSUES AN IO
  After IOs, the process issuing the IO will run LATER (when it is its turn)



- Here cpu ussage whould be 100%
- The CPU is never not it use. Process 0 will run until it completes or is killed and the Process 1 will start to run after that. There is no I/O and so there us no reason for either process to be blocked at all

In [10]:
%run process-run.py -cp -l 5:100,5:100

Time        PID: 0        PID: 1           CPU           IOs
  1        RUN:cpu         READY             1          
  2        RUN:cpu         READY             1          
  3        RUN:cpu         READY             1          
  4        RUN:cpu         READY             1          
  5        RUN:cpu         READY             1          
  6           DONE       RUN:cpu             1          
  7           DONE       RUN:cpu             1          
  8           DONE       RUN:cpu             1          
  9           DONE       RUN:cpu             1          
 10           DONE       RUN:cpu             1          

Stats: Total Time 10
Stats: CPU Busy 10 (100.00%)
Stats: IO Busy  0 (0.00%)



# Q2
Now run with these flags: ./process-run.py -l 4:100,1:0. These flags specify one process with 4 instructions (all to use the CPU), and one that simply issues an I/O and waits for it to be done.

- How long does it take to complete both processes? 
- Use -c and -p to find out if you were right.

In [11]:
%run process-run.py -l 4:100,1:0

Produce a trace of what would happen when you run these processes:
Process 0
  cpu
  cpu
  cpu
  cpu

Process 1
  io
  io_done

Important behaviors:
  System will switch when the current process is FINISHED or ISSUES AN IO
  After IOs, the process issuing the IO will run LATER (when it is its turn)



- I belive process 0 will be executed first and so the total type will be 6 ticks. 4 ticks using the cpu 1 tick to make the I/O call and 1 tick to return.

In [12]:
%run process-run.py -cp -l 4:100,1:0

Time        PID: 0        PID: 1           CPU           IOs
  1        RUN:cpu         READY             1          
  2        RUN:cpu         READY             1          
  3        RUN:cpu         READY             1          
  4        RUN:cpu         READY             1          
  5           DONE        RUN:io             1          
  6           DONE       BLOCKED                           1
  7           DONE       BLOCKED                           1
  8           DONE       BLOCKED                           1
  9           DONE       BLOCKED                           1
 10           DONE       BLOCKED                           1
 11*          DONE   RUN:io_done             1          

Stats: Total Time 11
Stats: CPU Busy 6 (54.55%)
Stats: IO Busy  5 (45.45%)



- LOL okay I missed in the readme where it said the defailt I/O time is 5 ticks. That makes since though. That makes since though because otherwise the I/O process will never be blocked and this question would make no since. Anyway except for the timing my basic prediction was correct. Process 0 executed first while Process 1 was scheduled. After Process 0 completed then process 1 started running, and took six ticks to complete.
- Take not that the CPU utilization is pretty low in this case

# Q3
Switch the order of the processes: -l 1:0,4:100. 
- What happens now?
- Does switching the order matter?
- Why? 
- (As always, use -c and -p to see if you were right)

In [13]:
%run process-run.py -l 1:0,4:100

Produce a trace of what would happen when you run these processes:
Process 0
  io
  io_done

Process 1
  cpu
  cpu
  cpu
  cpu

Important behaviors:
  System will switch when the current process is FINISHED or ISSUES AN IO
  After IOs, the process issuing the IO will run LATER (when it is its turn)



- Because Process 0 is now an I/O call that occurs first it becomes blocked and the Process 1 will start to run run while Process 0 is blocked. This will result in over all higher CPU utilization and less time required because the CPU won't be sitting waiting for the I/O to return.
- Yes the order mattered, although a more sefisticated process manager might have delt with that better
- The order mattered because it allowed for the process manager to kick off a time comsumer but CPU light process first and use that extra CPU time to run a more CPU intinsive process lower the overal time required and increasing total resource efficency.

- Predictions:
    - Total Time      : 7    Ticks
    - CPU Utilization : 85.7 %
    - I/O Utilization : 71.4 %

In [14]:
%run process-run.py -cp -l 1:0,4:100

Time        PID: 0        PID: 1           CPU           IOs
  1         RUN:io         READY             1          
  2        BLOCKED       RUN:cpu             1             1
  3        BLOCKED       RUN:cpu             1             1
  4        BLOCKED       RUN:cpu             1             1
  5        BLOCKED       RUN:cpu             1             1
  6        BLOCKED          DONE                           1
  7*   RUN:io_done          DONE             1          

Stats: Total Time 7
Stats: CPU Busy 6 (85.71%)
Stats: IO Busy  5 (71.43%)



Nice ( ͡° ͜ʖ ͡°)

# Q4
We’ll now explore some of the other flags. One important flag is `-S`, which determines how the system reacts when a process issues an I/O. With the flag set to SWITCH_ON_END, the system will NOT switch to another process while one is doing I/O, in stead waiting until the process is completely finished. 
- What happens when you run the following two processes (`-l 1:0,4:100 -c -S SWITCH_ON_END`), one doing I/O and the other doing CPU work?

- The results wil be identical to the results of Q2 but the process order will be flipped
- Predictions:
    - Total Time : 11 Ticks
    - CPU Utilization : 54.55 %
    - I/O Utilization : 45.45 %

In [17]:
%run process-run.py -l 1:0,4:100 -cp -S SWITCH_ON_END

Time        PID: 0        PID: 1           CPU           IOs
  1         RUN:io         READY             1          
  2        BLOCKED         READY                           1
  3        BLOCKED         READY                           1
  4        BLOCKED         READY                           1
  5        BLOCKED         READY                           1
  6        BLOCKED         READY                           1
  7*   RUN:io_done         READY             1          
  8           DONE       RUN:cpu             1          
  9           DONE       RUN:cpu             1          
 10           DONE       RUN:cpu             1          
 11           DONE       RUN:cpu             1          

Stats: Total Time 11
Stats: CPU Busy 6 (54.55%)
Stats: IO Busy  5 (45.45%)



# Q5
Now, run the same processes, but with the switching behavior set
to switch to another process whenever one is WAITING for I/O (`-l
1:0,4:100 -c -S SWITCH_ON_IO`). 
- What happens now? Use -c and -p to confirm that you are right.

- Predictions:
    - Total Time      : 7    Ticks
    - CPU Utilization : 85.7 %
    - I/O Utilization : 71.4 %

In [18]:
%run process-run.py -l 1:0,4:100 -cp -S SWITCH_ON_IO

Time        PID: 0        PID: 1           CPU           IOs
  1         RUN:io         READY             1          
  2        BLOCKED       RUN:cpu             1             1
  3        BLOCKED       RUN:cpu             1             1
  4        BLOCKED       RUN:cpu             1             1
  5        BLOCKED       RUN:cpu             1             1
  6        BLOCKED          DONE                           1
  7*   RUN:io_done          DONE             1          

Stats: Total Time 7
Stats: CPU Busy 6 (85.71%)
Stats: IO Busy  5 (71.43%)



# Q6
One other important behavior is what to do when an I/O completes. With `-I IO_RUN_LATER`, when an I/O completes, the process that issued it is not necessarily run right away; rather, whatever was running at the time keeps running. 
- What happens when you run this combination of processes? (Run `./process-run.py -l 3:0,5:100,5:100,5:100 -S SWITCH_ON_IO -I IO RUN_LATER -c -p`) 
- Are system resources being effectively utilized?

- Process 0 will run and issue and I/O call then Process 1 will run in full before the process 0 completes the first I/O then the first I/O will complete and a new I/O process will start and Processes 2 and 3 will run in the same fashion
- Prediction:
    - Total Time      : 21    Ticks
    - CPU Utilization : 100 %
    - I/O Utilization : 71.4 %

In [20]:
%run process-run.py -l 3:0,5:100,5:100,5:100 -S SWITCH_ON_IO -I IO_RUN_LATER -c -p

Time        PID: 0        PID: 1        PID: 2        PID: 3           CPU           IOs
  1         RUN:io         READY         READY         READY             1          
  2        BLOCKED       RUN:cpu         READY         READY             1             1
  3        BLOCKED       RUN:cpu         READY         READY             1             1
  4        BLOCKED       RUN:cpu         READY         READY             1             1
  5        BLOCKED       RUN:cpu         READY         READY             1             1
  6        BLOCKED       RUN:cpu         READY         READY             1             1
  7*         READY          DONE       RUN:cpu         READY             1          
  8          READY          DONE       RUN:cpu         READY             1          
  9          READY          DONE       RUN:cpu         READY             1          
 10          READY          DONE       RUN:cpu         READY             1          
 11          READY          DONE       RU

- okay so as soon at Process 0 became blocked then it ran the other 3 processies  then picked Process 0 back up. The reason it did this is because there was no down time between these processies for the I/O call to complete. The flag we set determined this. Because I/O calls are so time consuming it would make more since in this case to do as I described. I missunderstood the description of the flag

# Q7

Now run the same processes, but with `-I IO_RUN_IMMEDIATE` set, which immediately runs the process that issued the I/O. 
- How does this behavior differ? 
- Why might running a process that just completed an I/O again be a good idea?

- This will function as I described in the previous question
- I am not convivnced that it is always the best policy but in this case it in the right policy becuase the process will definietly be making an I/O call but I think a better policy would be to let any process make it's I/O call without delay (assuming that that resource isn't limited) that would allow for the most CPU utilization.
- Predictions:
    - Total Time      : 21    Ticks
    - CPU Utilization : 100 %
    - I/O Utilization : 71.4 %

In [21]:
%run process-run.py -l 3:0,5:100,5:100,5:100 -S SWITCH_ON_IO -I IO_RUN_IMMEDIATE -c -p

Time        PID: 0        PID: 1        PID: 2        PID: 3           CPU           IOs
  1         RUN:io         READY         READY         READY             1          
  2        BLOCKED       RUN:cpu         READY         READY             1             1
  3        BLOCKED       RUN:cpu         READY         READY             1             1
  4        BLOCKED       RUN:cpu         READY         READY             1             1
  5        BLOCKED       RUN:cpu         READY         READY             1             1
  6        BLOCKED       RUN:cpu         READY         READY             1             1
  7*   RUN:io_done          DONE         READY         READY             1          
  8         RUN:io          DONE         READY         READY             1          
  9        BLOCKED          DONE       RUN:cpu         READY             1             1
 10        BLOCKED          DONE       RUN:cpu         READY             1             1
 11        BLOCKED          DONE 

# Q8
Now run with some randomly generated processes: `-s 1 -l 3:50,3:50`
or `-s 2 -l 3:50,3:50` or `-s 3 -l 3:50,3:50`. See if you can
predict how the trace will turn out. 
- What happens when you use the flag `-I IO_RUN_IMMEDIATE` vs. `-I IO_RUN_LATER`? 
- What happens when you use `-S SWITCH_ON_IO` vs. `-S SWITCH_ON_END`?

- I think in general the CPU utilization and will be higher and time requiemenet will be lower for the IO_RUN_IMMEDIATE condition.
- Similarly I feel that the SWITCH_ON_IO condition will result in generally higher CPU utilization

In [37]:
%run process-run.py -s 3 -l 3:50,3:50 -cp -I IO_RUN_IMMEDIATE -S SWITCH_ON_IO

Time        PID: 0        PID: 1           CPU           IOs
  1        RUN:cpu         READY             1          
  2         RUN:io         READY             1          
  3        BLOCKED        RUN:io             1             1
  4        BLOCKED       BLOCKED                           2
  5        BLOCKED       BLOCKED                           2
  6        BLOCKED       BLOCKED                           2
  7        BLOCKED       BLOCKED                           2
  8*   RUN:io_done       BLOCKED             1             1
  9*         READY   RUN:io_done             1          
 10          READY        RUN:io             1          
 11        RUN:cpu       BLOCKED             1             1
 12           DONE       BLOCKED                           1
 13           DONE       BLOCKED                           1
 14           DONE       BLOCKED                           1
 15           DONE       BLOCKED                           1
 16*          DONE   RUN:io_done        

In [36]:
%run process-run.py -s 3 -l 3:50,3:50 -cp -I IO_RUN_LATER -S SWITCH_ON_IO

Time        PID: 0        PID: 1           CPU           IOs
  1        RUN:cpu         READY             1          
  2         RUN:io         READY             1          
  3        BLOCKED        RUN:io             1             1
  4        BLOCKED       BLOCKED                           2
  5        BLOCKED       BLOCKED                           2
  6        BLOCKED       BLOCKED                           2
  7        BLOCKED       BLOCKED                           2
  8*   RUN:io_done       BLOCKED             1             1
  9*       RUN:cpu         READY             1          
 10           DONE   RUN:io_done             1          
 11           DONE        RUN:io             1          
 12           DONE       BLOCKED                           1
 13           DONE       BLOCKED                           1
 14           DONE       BLOCKED                           1
 15           DONE       BLOCKED                           1
 16           DONE       BLOCKED            

In [35]:
%run process-run.py -s 3 -l 3:50,3:50 -cp -I IO_RUN_IMMEDIATE -S SWITCH_ON_END

Time        PID: 0        PID: 1           CPU           IOs
  1        RUN:cpu         READY             1          
  2         RUN:io         READY             1          
  3        BLOCKED         READY                           1
  4        BLOCKED         READY                           1
  5        BLOCKED         READY                           1
  6        BLOCKED         READY                           1
  7        BLOCKED         READY                           1
  8*   RUN:io_done         READY             1          
  9        RUN:cpu         READY             1          
 10           DONE        RUN:io             1          
 11           DONE       BLOCKED                           1
 12           DONE       BLOCKED                           1
 13           DONE       BLOCKED                           1
 14           DONE       BLOCKED                           1
 15           DONE       BLOCKED                           1
 16*          DONE   RUN:io_done            

In [34]:
%run process-run.py -s 3 -l 3:50,3:50 -cp -I IO_RUN_LATER -S SWITCH_ON_END

Time        PID: 0        PID: 1           CPU           IOs
  1        RUN:cpu         READY             1          
  2         RUN:io         READY             1          
  3        BLOCKED         READY                           1
  4        BLOCKED         READY                           1
  5        BLOCKED         READY                           1
  6        BLOCKED         READY                           1
  7        BLOCKED         READY                           1
  8*   RUN:io_done         READY             1          
  9        RUN:cpu         READY             1          
 10           DONE        RUN:io             1          
 11           DONE       BLOCKED                           1
 12           DONE       BLOCKED                           1
 13           DONE       BLOCKED                           1
 14           DONE       BLOCKED                           1
 15           DONE       BLOCKED                           1
 16*          DONE   RUN:io_done            